In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
train = pd.read_json('../file/train.json', encoding='utf-8')

In [ ]:
validation = pd.read_json('../file/val.json', encoding='utf-8')

In [ ]:
# validation.head(100).songs.apply(lambda e: len(e))

In [ ]:
song_meta = pd.read_json('../file/song_meta.json', encoding='utf-8')

In [ ]:
Counter([len(_) for _ in train.tags]).most_common()

In [ ]:
tag_count = Counter([__ for _ in train.tags for __ in _])

In [ ]:
len(tag_count)

In [ ]:
len(list(filter(lambda e: e[1] > 10, tag_count.items())))

In [ ]:
tag_count.most_common(20)

## tag들로 word2vec 학습이 될까?

In [ ]:
from gensim.models import Word2Vec

In [ ]:
w2v = Word2Vec(train.tags, min_count=10, window=2)

In [ ]:
w2v.wv.most_similar(positive=['힙합'], negative=['감성'])

In [ ]:
w2v.wv.most_similar('감성')

In [ ]:
w2v.wv.most_similar('락')

In [ ]:
w2v.wv.most_similar('비')

In [ ]:
w2v.wv.most_similar('매장음악')

In [ ]:
w2v.wv['매장음악']

## playlist제목으로 word2vec이 될까?

In [ ]:
train.columns

In [ ]:
from konlpy.tag import Komoran

In [ ]:
pos = Komoran()
# pos = Kkma()

In [ ]:
pos.pos('안녕하세요 현수씨')

In [ ]:
train['plyst_title_pos'] = train.plylst_title.apply(lambda e: [_[0] for _ in pos.pos(e)
                                                              if not _[1].startswith('J') 
                                                              and len(_[0])>1])

In [ ]:
validation.plylst_title

In [ ]:
def get_pos(e):
    try:
        return [_[0] for _ in pos.pos(e) if not _[1].startswith('J') and len(_[0])>1]
    except:
        return []

In [ ]:
validation['plyst_title_pos'] = validation.plylst_title.apply(lambda e: get_pos(e))

In [ ]:
w2v_song = Word2Vec(train.plyst_title_pos, min_count=10)

In [ ]:
w2v_song.wv.most_similar('노래')

In [ ]:
w2v_song.wv.most_similar(['사랑', '노래', '이별'])

In [ ]:
w2v_song.wv.most_similar('강아지')

#  장르

In [ ]:
song_meta = pd.read_json('../file/song_meta.json', encoding='utf-8')

In [ ]:
import json
import itertools

In [ ]:
with open('../file/genre_gn_all.json', encoding='utf-8') as f:
    genre = json.load(f)

In [ ]:
genre = {key: (genre[key[:-1]+'0'] if key[-2:] =='01' else value) for key, value in genre.items()}

In [ ]:
song_meta['genre_list'] = song_meta.song_gn_dtl_gnr_basket

In [ ]:
song_meta['song_gn_word'] = song_meta.genre_list.apply(lambda e: list(map(lambda x: genre[x], filter(lambda y: y in genre, e))))

In [ ]:
song_meta['song_gn_word_split'] = song_meta.song_gn_word.apply(lambda e: list(itertools.chain(*map(lambda x: x.split('/'), e))))

In [ ]:
song_meta_dropna = song_meta.dropna(subset=['song_gn_word_split'])

In [ ]:
song_id_genre = dict(zip(song_meta_dropna['id'], song_meta_dropna['song_gn_word_split']))

In [ ]:
train['genre'] = train.songs.apply(lambda e: Counter(itertools.chain(*list(map(lambda x: song_id_genre[x], e)))).most_common(5))
validation['genre'] = validation.songs.apply(lambda e: Counter(itertools.chain(*list(map(lambda x: song_id_genre[x], e)))).most_common(5))

In [ ]:
train['genre_kor'] = train['genre'].apply(lambda e: list(list(zip(*e))[0]) if e else [])
validation['genre_kor'] = validation['genre'].apply(lambda e: list(list(zip(*e))[0]) if e else [])

In [ ]:
corpus = train.tags + train.plyst_title_pos + train.genre_kor

In [ ]:
def cleansing(string):
    kor_alpha = re.compile('[ㄱ-ㅎ]')
    etc = re.compile(re.escape("',./;[]_-=()*&^%$#@!~"))
    re_list = [kor_alpha, etc]
    for pttn in re_list:
        string = pttn.sub("", string)
    string = string.lower()
    return string

In [ ]:
corpus = list(map(lambda e: [cleansing(word) for word in e if cleansing(word)], corpus))

In [ ]:
train.to_pickle('../file/train_pos.pickle')
validation.to_pickle('../file/validataion_pos.pickle')

In [ ]:
w2v_all = Word2Vec(train.tags, min_count=10, window=10, size=200)

In [ ]:
w2v.save('../file/w2v.model')

- genre를 추가해서 태그랑 같이 학습해도될듯